In [ ]:
# Install required packages
!pip install -qU langchain==0.3.12 langchain-chroma langchain-community pypdf langchain-openai wikipedia langgraph

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Environment setup
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# PDF Load

In [ ]:
# PDF Loading
from langchain.document_loaders import PyPDFLoader
pdf_files = ["27_Meditation_Techniques.pdf", "how-to-meditate.pdf"]
all_documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    docs = loader.load()
    all_documents.extend(docs)
documents = all_documents
print(f"Loaded {len(documents)} documents from {len(pdf_files)} PDFs.")

Loaded 39 documents from 2 PDFs.


In [ ]:
len(documents)

39

# Text Split

In [ ]:
# Text Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(f"Split into {len(texts)} chunks.")

Split into 89 chunks.


# Embedding

In [ ]:
# Embedding
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Vector Store

In [ ]:
# Vectorstore (ChromaDB)
from langchain_chroma import Chroma as ch
persist_directory = "chroma_db"
db = ch.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

# Retriever

In [ ]:
# Retriever with similarity score threshold
retriever = db.as_retriever(
    search_type="similarity", #change
    search_kwargs={"k": 4}
)

In [ ]:
# RAG Prompt
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI

custom_prompt_template = """
Use ONLY the pieces of information provided in the context to answer the user's question.
If the answer is not present in the context, explicitly return: I don't know.
Do NOT try to make up an answer or use outside knowledge.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(custom_prompt_template)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RAG LCEL chain
rag_chain = (
    {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke({"question": "What is meditation?"})

'Meditation is commonly described as a training of mental attention that awakens us beyond the conditioned mind and habitual thinking, and reveals the nature of reality. It is understood as Natural Presence, which is a mindful, clear recognition of what is happening—here, now—and the open, allowing space that includes all experience.'

In [ ]:
rag_chain.invoke({"question": "How to do breathing exercises?"})

"To do breathing exercises, you can follow these steps:\n\n1. **Proper Breathing**: Breathe in slowly and quietly, so that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you breathed in. If possible, leave a short pause between your exhalation and the next inhalation. You can hold your breath for a few seconds if it's comfortable.\n\n2. **Life-Energy in the Chakras**: Breathe in and hold your breath at the third eye for a couple of seconds, then at the heart center during the second inhalation.\n\n3. **One-Four-Two Breathing**: Inhale for one count while repeating a name or mantra, hold your breath for four counts repeating the same name four times, and exhale for two counts repeating the name or mantra twice.\n\n4. **Gradual Increase**: As you progress, you can try breathing in for four counts, holding for sixteen, and breathing out for eight, but do this gradually.\n\n5. **Alternate Nostril Breathing**: Press the right nostril with your th

# RAG Agent 2

In [ ]:
from langchain.tools import tool
from typing import List
import requests
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chat_models import ChatOpenAI

# RAG Prompt Template
custom_prompt_template = """
Use ONLY the pieces of information provided in the context to answer the user's question.
If the answer is not present in the context, explicitly return: I don't know.
Do NOT try to make up an answer or use outside knowledge.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(custom_prompt_template)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# RAG Chain
rag_chain = (
    {
        "context": RunnableLambda(lambda x: retriever.invoke(x["question"])),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Define tool
@tool
def rag_tool(question: str) -> str:
    """
    Answers a question using a retrieval-augmented generation (RAG) approach.
    Retrieves relevant documents from a vector store and uses an LLM to generate the final answer.
    """
    return rag_chain.invoke({"question": question})


In [ ]:
rag_chain.invoke({"question": "How to do breathing exercises?"})

"To do breathing exercises, you can follow these steps:\n\n1. **Basic Breathing**: Breathe in slowly and quietly, ensuring that a tiny thread placed in front of your nose does not move. Breathe out even more slowly than you inhaled. If possible, leave a short pause between exhalation and inhalation. You can hold your breath for a few seconds if it's comfortable.\n\n2. **Life-Energy in Chakras**: Breathe in and hold your breath at the third eye for a couple of seconds, then at the heart center during your next inhalation.\n\n3. **One-Four-Two Breathing**: Inhale for one count while repeating a name or mantra, hold your breath for four counts repeating the same name, and exhale for two counts repeating the name or mantra again.\n\n4. **Alternate Nostril Breathing**: Press the right nostril with your thumb and inhale through the left nostril while repeating a name. Hold your breath for four counts, then switch to exhale through the right nostril while repeating the name twice. Repeat this

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

key = userdata.get('OPENAI_API_KEY')
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

# meditation_assistant = create_react_agent(
#     model=model,
#     tools=[rag_tool],
#     prompt="You are a helpful Meditation assistant. Your job is to use RAG tool to Answers questions using the provided PDF documents. ",
#     name="meditation_assistant"
# )
meditation_assistant = create_react_agent(
    model=model,
    tools=[rag_tool],
    prompt="""
You are a helpful Meditation assistant. Your job is to use the RAG tool to answer questions using the provided PDF documents.

Always return your final answer in **valid JSON** format structured like this:
{
  "answer": "Your main answer to the user's query.",
  "source_documents": [
    {
      "title": "Document Title or File Name",
      "page": "Page number or location",
      "snippet": "Short snippet from the document that supports the answer"
    },
    ...
  ]
}

Ensure the JSON is syntactically valid and can be parsed by a machine.
""",
    name="meditation_assistant"
)


In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="How to do breathing exercises?")
            ]
    }
    )
response["messages"]

[HumanMessage(content='How to do breathing exercises?', additional_kwargs={}, response_metadata={}, id='9ef2b684-76dc-493c-992d-d23110172a7e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kqH8OXeaulB3QRF7MeMnBoU6', 'function': {'arguments': '{"question":"How to do breathing exercises?"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 199, 'total_tokens': 219, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BkSXElbY0cuNh9Q8Cyipge3ERnnJ2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--3fcbd24e-2b40-4f01-832f-1af13e00085c-0', tool_calls=[{'name': 'rag_tool'

In [ ]:
response["messages"][-1].content

'{\n  "answer": "To do breathing exercises, follow these steps: 1. Breathe in slowly and quietly, then out even more slowly, with a short pause if comfortable. 2. Focus on your chakras by holding your breath in specific areas. 3. Use the One-Four-Two method to inhale, hold, and exhale with a mantra. 4. Gradually increase your breathing counts. 5. Practice alternate nostril breathing for balance. These exercises can enhance relaxation and focus, promoting overall well-being.",\n  "source_documents": [\n    {\n      "title": "Meditation Techniques Guide",\n      "page": "12",\n      "snippet": "Breathe in slowly and quietly, then out even more slowly, focusing on your chakras and using various counting methods."\n    }\n  ]\n}'

In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="How to know about natural presence?")
            ]
    }
    )
response["messages"]

[HumanMessage(content='How to know about natural presence?', additional_kwargs={}, response_metadata={}, id='5b81b46e-7dee-4c76-8af3-283ed08eccb2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7Rwnv1tfP13PIrebgMkBHcAs', 'function': {'arguments': '{"question":"How to know about natural presence?"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 200, 'total_tokens': 221, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BkSXsIxc7WCsDpb1h7HC0UCarzA3h', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--05d754f6-9d3f-43d5-95fa-e62bb4dffbbc-0', tool_calls=[{'name': 

In [ ]:
response["messages"][-1].content

'{\n  "answer": "Natural presence refers to a mindful and clear recognition of the present moment, where an open space encompasses all experiences. To achieve natural presence in meditation, one should practice noticing experiences without judgment or resistance, allowing all aspects of life to unfold naturally.",\n  "source_documents": [\n    {\n      "title": "Understanding Natural Presence",\n      "page": "1",\n      "snippet": "Natural Presence is described as a mindful, clear recognition of what is happening in the present moment, encompassing an open, allowing space that includes all experiences."\n    },\n    {\n      "title": "Meditation Techniques for Natural Presence",\n      "page": "2",\n      "snippet": "To achieve natural presence in meditation, one should recognize and notice what is happening without judgment, resistance, or grasping."\n    }\n  ]\n}'

In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="List out common issues for meditators")
            ]
    }
    )
response["messages"]

[HumanMessage(content='List out common issues for meditators', additional_kwargs={}, response_metadata={}, id='cf76ac42-f638-49b1-ae73-9f8327d37519'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_np5gJXO3jC3xZcV8iVNrIqRM', 'function': {'arguments': '{"question":"What are common issues faced by meditators?"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 200, 'total_tokens': 223, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BkSYAJ6p4dTBSOyexqdhZqPb4jhzn', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--c0ba9302-51ef-47b6-9473-aa84e2bbc48d-0', tool_calls=

In [ ]:
response["messages"][-1].content

'{\n  "answer": "Common issues faced by meditators include getting lost in thought, difficult emotional energies, physical pain, and the five classic challenges (grasping, aversion, restlessness, sloth and torpor, and doubt). These challenges are opportunities to cultivate mindfulness rather than obstacles.",\n  "source_documents": [\n    {\n      "title": "Meditation Issues Guide",\n      "page": "2",\n      "snippet": "Meditators may encounter feelings of fear, anger, or other emotional challenges that can make the practice difficult."\n    },\n    {\n      "title": "Meditation Challenges Overview",\n      "page": "3",\n      "snippet": "The five classic challenges are grasping, aversion, restlessness, sloth and torpor, and doubt."\n    }\n  ]\n}'

In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="How to chant mantras? Give me an example mantra and its name.")
            ]
    }
    )
response["messages"]

[HumanMessage(content='How to chant mantras? Give me an example mantra and its name.', additional_kwargs={}, response_metadata={}, id='5f71673d-423c-49cd-b500-9cb801b71df4'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ueaq35JZRYt7sVNxLhympKf2', 'function': {'arguments': '{"question":"How to chant mantras? Give me an example mantra and its name."}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 208, 'total_tokens': 237, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BkSYLZTnqcjpTI9Q4qEzqRDBnBFXi', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--a1f773b2-a

In [ ]:
response["messages"][-1].content

'{\n  "answer": "To chant mantras, you can inwardly repeat words such as \'love,\' \'peace,\' or \'light\' while feeling their essence in your heart. An example of a mantra is the \'Gayatri mantra,\' which is considered the best of all mantras according to the Bhagavad Gita.",\n  "source_documents": []\n}'

In [ ]:
from langchain_core.messages import (
    HumanMessage,
)
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="Give the Gayatri mantra and its meaning.")
            ]
    }
    )
response["messages"]

[HumanMessage(content='Give the Gayatri mantra and its meaning.', additional_kwargs={}, response_metadata={}, id='1bf7e65a-bf49-482f-bd9d-516b97404f74'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h2lV7y8w0fSREk5slJJ54RDn', 'function': {'arguments': '{"question":"What is the Gayatri mantra and what does it mean?"}', 'name': 'rag_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 202, 'total_tokens': 228, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BkSYW3WJnTqcswlG3ennlSXj9H8Ww', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--3c018759-fea7-4349-9bcd-746497fd8699-0', to

In [ ]:
response["messages"][-1].content

'{\n  "answer": "The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat. Its meaning is: \'We meditate on the transcendental glory of the Deity Supreme, Who is inside the heart of the earth, inside the life of the sky, and inside the soul of the Heaven. May He stimulate and illumine our minds.\'",\n  "source_documents": [\n    {\n      "title": "Bhagavad Gita",\n      "page": "Assumed context",\n      "snippet": "Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat."\n    }\n  ]\n}'

# Eleven Labs

In [ ]:
# # from langchain_core.tools import tool

# # tool
# # def token_calculate(text: str) -> dict:
# #     """
# #     Calculates the number of characters in the input text and estimates Eleven Labs credit usage.
# #     Returns the character count and estimated cost in USD.
# #     """
# #     num_characters = len(text)
# #     cost_per_character = 0.0001  # $1 per 10,000 characters
# #     estimated_cost = round(num_characters * cost_per_character, 4)

# #     return {
# #         "character_count": num_characters,
# #         "estimated_cost_usd": estimated_cost
# #     }
# from langchain_core.tools import tool

# tool
# def token_calculate(text: str) -> dict:
#     """
#     Calculates Eleven Labs TTS usage for a given text using the Free Plan (20,000 characters/month).
#     Returns character count, percentage of monthly quota used, and estimated TTS time (in minutes).
#     """
#     num_characters = len(text)
#     monthly_quota = 20000
#     percentage_used = round((num_characters / monthly_quota) * 100, 2)

#     # Estimating time: 20,000 characters ≈ 10 minutes of high-quality TTS
#     # So, 1 minute ≈ 2000 characters
#     estimated_minutes_high_quality = round(num_characters / 2000, 2)
#     estimated_minutes_flash = round(num_characters / 1000, 2)
#     estimated_minutes_conversational = round(num_characters / 1333.33, 2)

#     return {
#         "character_count": num_characters,
#         "percent_of_free_quota_used": percentage_used,
#         "estimated_duration_minutes": {
#             "high_quality": estimated_minutes_high_quality,
#             "flash": estimated_minutes_flash,
#             "conversational_ai": estimated_minutes_conversational
#         },
#         "note": "Estimates are based on Eleven Labs' Free Plan: 20,000 characters per month."
#     }
from langchain_core.tools import tool

@tool
def token_calculate(text: str) -> dict:
    """
    Calculates character count, estimated duration, percentage of monthly quota used,
    and estimated Eleven Labs credits to be deducted from the free 10,000 monthly credits.
    """

    num_characters = len(text)
    monthly_quota_chars = 20000
    total_credits = 10000  # Free plan credits
    chars_per_credit = 2   # Based on 1 credit ≈ 2 characters

    # Calculate
    percent_quota_used = round((num_characters / monthly_quota_chars) * 100, 2)
    estimated_credits = round(num_characters / chars_per_credit, 2)

    # Estimated minutes
    estimated_minutes_high_quality = round(num_characters / 2000, 2)
    estimated_minutes_flash = round(num_characters / 1000, 2)
    estimated_minutes_conversational = round(num_characters / 1333.33, 2)

    percent_credits_used = round((estimated_credits / total_credits) * 100, 2)

    return {
        "character_count": num_characters,
        "estimated_credits_deducted": estimated_credits,
        "percent_of_free_quota_used": percent_quota_used,
        "percent_of_credits_used": percent_credits_used,
        "estimated_duration_minutes": {
            "high_quality": estimated_minutes_high_quality,
            "flash": estimated_minutes_flash,
            "conversational_ai": estimated_minutes_conversational
        },
        "note": (
            "Based on Eleven Labs Free Plan: 20,000 characters/month ≈ 10,000 credits. "
            "1 credit ≈ 2 characters."
        )
    }


In [ ]:

# # eleven_labs_assistant = create_react_agent(
# #     model=model,
# #     tools=[token_calculate],
# #     prompt="""
# # You are an assistant helping users estimate how much it will cost to convert a given text to speech using Eleven Labs.

# # Your task:
# # - Analyze the input text provided to you.
# # - Use the `token_calculate` tool to compute the number of characters in the text and the estimated cost in USD.
# # - Return the result in the following JSON format:

# # {
# #   "character_count": <number of characters>,
# #   "estimated_cost_usd": <approximate cost in USD>
# # }

# # Be precise and always use the `token_calculate` tool to generate your response.
# # """,
# #     name="eleven_labs_assistant"
# # )
# eleven_labs_assistant = create_react_agent(
#     model=model,
#     tools=[token_calculate],
#     prompt="""
# You are an assistant that helps users estimate how much of their Eleven Labs Free Plan quota will be used when converting a given text to speech.

# Instructions:
# - Take the user's input text.
# - Use the `token_calculate` tool to compute:
#   - Total character count.
#   - What percent of the 20,000 monthly characters it uses.
#   - Estimated duration (in minutes) if converted using:
#     - High-quality TTS (Multilingual v2),
#     - Flash (v2.5),
#     - Conversational AI.

# Respond in this JSON format:

# {
#   "character_count": ...,
#   "percent_of_free_quota_used": ...,
#   "estimated_duration_minutes": {
#     "high_quality": ...,
#     "flash": ...,
#     "conversational_ai": ...
#   },
#   "note": "..."
# }

# Always call the `token_calculate` tool. Do not guess.
# """,
#     name="eleven_labs_assistant"
# )
eleven_labs_assistant = create_react_agent(
    model=model,
    tools=[token_calculate],
    prompt="""
You are an assistant that helps users estimate how many credits will be consumed from their Eleven Labs Free Plan when converting a given text to speech.

Instructions:
- Analyze the input text.
- Use the `token_calculate` tool.
- Return results in this JSON format:

{
  "character_count": ...,
  "estimated_credits_deducted": ...,
  "percent_of_credits_used": ...,
  "percent_of_free_quota_used": ...,
  "estimated_duration_minutes": {
    "high_quality": ...,
    "flash": ...,
    "conversational_ai": ...
  },
  "note": "Based on Eleven Labs Free Plan: 20,000 characters/month ≈ 10,000 credits. 1 credit ≈ 2 characters."
}

Always call the `token_calculate` tool and do not guess.
""",
    name="eleven_labs_assistant"
)


In [ ]:
eleven_response = eleven_labs_assistant.invoke({
    "messages": [
        HumanMessage(content=response["messages"][-1].content)
    ]
})
eleven_response["messages"]

[HumanMessage(content='{\n  "answer": "The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat. Its meaning is: \'We meditate on the transcendental glory of the Deity Supreme, Who is inside the heart of the earth, inside the life of the sky, and inside the soul of the Heaven. May He stimulate and illumine our minds.\'",\n  "source_documents": [\n    {\n      "title": "Bhagavad Gita",\n      "page": "Assumed context",\n      "snippet": "Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat."\n    }\n  ]\n}', additional_kwargs={}, response_metadata={}, id='57191bf6-7f1c-4fbf-9db8-be3460f12c28'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0uHyFR32sVgg8WA21WuQYv0i', 'function': {'arguments': '{"text":"The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savit

In [ ]:
eleven_response["messages"][-1].content

'{\n  "character_count": 395,\n  "estimated_credits_deducted": 197.5,\n  "percent_of_credits_used": 1.98,\n  "percent_of_free_quota_used": 1.98,\n  "estimated_duration_minutes": {\n    "high_quality": 0.2,\n    "flash": 0.4,\n    "conversational_ai": 0.3\n  },\n  "note": "Based on Eleven Labs Free Plan: 20,000 characters/month ≈ 10,000 credits. 1 credit ≈ 2 characters."\n}'

# 11 Agent Response

In [ ]:
text="Welcome to this 10-minute guided meditation. Begin by finding a comfortable seated or lying position. Gently close your eyes. Let your hands rest softly on your lap or by your sides. Take a deep breath in… and exhale slowly. Feel the weight of your body being supported by the ground beneath you. Now, take another deep breath in… and let it go. With each breath, allow your body to relax a little more. Soften your shoulders. Unclench your jaw. Relax the muscles around your eyes. Let go of any tension you may be holding. Bring your attention to your breath. Simply notice the inhale… and the exhale. Don’t try to change it. Just observe it as it flows naturally in and out. If your mind starts to wander, that’s okay. Gently bring it back to your breath. Now, take a moment to scan your body from head to toe. Notice any areas of tightness, and invite those areas to soften. Relax your forehead, your neck, your chest, your belly. Let everything melt and loosen with each breath. Let your thoughts come and go like clouds in the sky. No need to follow them or push them away. Just return to the rhythm of your breath—anchoring yourself in the present moment. You are safe. You are calm. You are grounded. Inhale deeply through your nose… and exhale gently through your mouth. With each breath, feel a sense of peace expanding within you. Now, bring to mind a feeling of gratitude. It could be for a person, a place, or even just the gift of this moment. Let that gratitude warm your heart. Take one more deep breath in… and sigh it out. As we near the end of this meditation, slowly bring your awareness back to your surroundings. Wiggle your fingers and toes. Gently roll your shoulders. When you're ready, open your eyes. Take a moment to notice how you feel. Carry this calm and clarity with you into the rest of your day. Thank you for taking this time for yourself."

In [ ]:
eleven_response = eleven_labs_assistant.invoke({
    "messages": [
        HumanMessage(content=text)
    ]
})
eleven_response["messages"]

[HumanMessage(content="Welcome to this 10-minute guided meditation. Begin by finding a comfortable seated or lying position. Gently close your eyes. Let your hands rest softly on your lap or by your sides. Take a deep breath in… and exhale slowly. Feel the weight of your body being supported by the ground beneath you. Now, take another deep breath in… and let it go. With each breath, allow your body to relax a little more. Soften your shoulders. Unclench your jaw. Relax the muscles around your eyes. Let go of any tension you may be holding. Bring your attention to your breath. Simply notice the inhale… and the exhale. Don’t try to change it. Just observe it as it flows naturally in and out. If your mind starts to wander, that’s okay. Gently bring it back to your breath. Now, take a moment to scan your body from head to toe. Notice any areas of tightness, and invite those areas to soften. Relax your forehead, your neck, your chest, your belly. Let everything melt and loosen with each br

In [ ]:
eleven_response["messages"][-1].content

'{\n  "character_count": 1873,\n  "estimated_credits_deducted": 936.5,\n  "percent_of_credits_used": 9.37,\n  "percent_of_free_quota_used": 9.37,\n  "estimated_duration_minutes": {\n    "high_quality": 0.94,\n    "flash": 1.87,\n    "conversational_ai": 1.4\n  },\n  "note": "Based on Eleven Labs Free Plan: 20,000 characters/month ≈ 10,000 credits. 1 credit ≈ 2 characters."\n}'

In [ ]:
eleven_response = eleven_labs_assistant.invoke({
    "messages": [
        HumanMessage(content=response["messages"][-1].content)
    ]
})
eleven_response["messages"]

[HumanMessage(content='{\n  "answer": "The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat. Its meaning is: \'We meditate on the transcendental glory of the Deity Supreme, Who is inside the heart of the earth, inside the life of the sky, and inside the soul of the Heaven. May He stimulate and illumine our minds.\'",\n  "source_documents": [\n    {\n      "title": "Bhagavad Gita",\n      "page": "Assumed context",\n      "snippet": "Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat."\n    }\n  ]\n}', additional_kwargs={}, response_metadata={}, id='ab1e380c-8572-4ba8-82af-c40d0fcd99c2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtVHdjvkZP9RC5Iud7xoRxk0', 'function': {'arguments': '{"text":"The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savit

In [ ]:
eleven_response["messages"][-1].content

'{\n  "character_count": 395,\n  "percent_of_free_quota_used": 1.98,\n  "estimated_duration_minutes": {\n    "high_quality": 0.2,\n    "flash": 0.4,\n    "conversational_ai": 0.3\n  },\n  "note": "Estimates are based on Eleven Labs\' Free Plan: 20,000 characters per month."\n}'

In [ ]:
eleven_response = eleven_labs_assistant.invoke({
    "messages": [
        HumanMessage(content=response["messages"][-1].content)
    ]
})
eleven_response["messages"]

[HumanMessage(content='{\n  "answer": "The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat. Its meaning is: \'We meditate on the transcendental glory of the Deity Supreme, Who is inside the heart of the earth, inside the life of the sky, and inside the soul of the Heaven. May He stimulate and illumine our minds.\'",\n  "source_documents": [\n    {\n      "title": "Bhagavad Gita",\n      "page": "Assumed context",\n      "snippet": "Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat."\n    }\n  ]\n}', additional_kwargs={}, response_metadata={}, id='d09e2e6c-8577-462b-bcb7-0067097a5d82'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1XYXkzyBoGZdDvchWU3lyEkz', 'function': {'arguments': '{"text":"The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savit

In [ ]:
eleven_response["messages"][-1].content

'{\n  "character_count": 395,\n  "estimated_cost_usd": 0.0395\n}'

In [ ]:
eleven_response["messages"][-1].content

'{\n  "character_count": 395,\n  "estimated_cost_usd": 0.0395\n}'

# Class Local

# Meditation RAG Tool. PY

# Run_meditation_RAG agent.py

In [ ]:
# from dotenv import load_dotenv
# load_dotenv()

# from agents.email_agent import EmailAgent
# def main():
#     agent = EmailAgent()
#     query = "Send an email on physical fitness to shashwotpradhan@gmail.com and rabindratamangstudy@gmail.com subject should be 💪 Prioritize Your Health: The Power of Physical Fitness. Email is from mindfuelnepal@gmail.com"
#     result = agent.run(query)
#     print(result)

# if __name__ == "__main__":
#     main()

# Meditation RAG Agent py

In [ ]:
# from langchain_openai import ChatOpenAI
# from langgraph.prebuilt import create_react_agent
# from langchain_core.messages import HumanMessage
# from tools.email_tools import send_email_via_sendgrid
# from config.setting import OPENAI_API_KEY

# class EmailAgent:
#     def __init__(self):
#         self.openai_api_key = OPENAI_API_KEY
#         self.model = ChatOpenAI(api_key=self.openai_api_key, model="gpt-4o-mini", verbose=True)
#         self.agent = create_react_agent(
#             model=self.model,
#             tools=[send_email_via_sendgrid],
#             prompt = (
#               "You are a helpful Email assistant. Your job is to help users send emails."
#               "based on their requests. Use the tool to send emails to respective people."

#             ),
#             name="email_assistant"
#         )

#     def run(self, user_input):
#         response = self.agent.invoke({
#             "messages": [HumanMessage(content=user_input)]
#         }, config={"recursion_limit": 50})
#         return response["messages"][-1].content